<a href="https://colab.research.google.com/github/lxguan1/EECS583_Unroll_ML/blob/main/model/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader    # dataset representation and loading
import torch.nn as nn                     # neural networks
import torch.nn.functional as F           # layers, activations and more
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.
import numpy as np
import csv
import pdb

In [2]:
device = None
if torch.cuda.is_available():
    print("Using the GPU")
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')
    print("NOTE: Using CPU")

Using the GPU


In [3]:
class CSVDataset(Dataset): 
    # 9 feats: trip count, num ops, num operands, num mem ops, num fops, num branches, est resmii, frequent path length, depth of loop

    def __init__(self, csv_path):
        csv_reader = csv.reader(open(csv_path, 'r'), delimiter=',')

        # line = [prob_name, f1, ... , f9, rank1, ..., rank8]
        self.csv_lines = []
        for line in csv_reader:
            row = [float(item) for item in line[1:10]]
            row.append(int(line[11]))
            self.csv_lines.append(row)
    
    def __len__(self):
        return len(self.csv_lines)
    
    def __getitem__(self, idx):
        line = self.csv_lines[idx]
        X = torch.FloatTensor(line[:-1])
        y = torch.zeros(8)
        y[line[-1] - 1] = 1
        return X, y

In [4]:
class MLP(nn.Module):
    def __init__(self,input_size,hidden_dim_1,hidden_dim_2,num_classes,drop_prob=0.5):
      super(MLP, self).__init__()
      #Put GeneModel architecture here (WITHOUT THE FINAL LAYERS)
      self.hidden_dim_1=hidden_dim_1
      self.hidden_dim_2=hidden_dim_2
      self.num_classes=num_classes
      
      #self.flantten=torch.flatten()
      self.fc_1=nn.Linear(input_size,hidden_dim_1)
      self.fc_2=nn.Linear(hidden_dim_1,hidden_dim_2)
      self.fc_3=nn.Linear(hidden_dim_2,num_classes)

      self.relu = nn.ReLU()
      self.dropout = nn.Dropout(p=drop_prob)

    def forward(self, x):
      #Put GeneModel architecture here (WITHOUT THE FINAL LAYERS)
      x=x.float()
      x=self.fc_1(x)
      x=self.dropout(x)
      x=self.relu(x)
      x=self.fc_2(x)
      x=self.dropout(x)
      x=self.relu(x)
      x=self.fc_3(x)
      return x
    
    def train_model(self, train_loader, optimizer, criterion):
      train_loss = 0.0
      for X, y in train_loader:
        # Setup 
        X = X.cuda()
        y = y.cuda()
        optimizer.zero_grad()

        # Send batch through 
        pred = mlp(X)
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

      return train_loss

    def validate_model(self, val_loader, criterion):
      val_loss_sum = 0.0
      for X, y in val_loader:
        X = X.cuda()
        y = y.cuda()
        
        pred = mlp(X)
        val_loss = criterion(pred, y)
        val_loss_sum += val_loss.item()
      
      return val_loss_sum

In [7]:
# Setup model and hyper parameters
mlp = MLP(input_size=9, hidden_dim_1=8, hidden_dim_2=8, num_classes=8, drop_prob=0.0).cuda()
lr = 0.001
optimizer = optim.Adam(mlp.parameters(), lr=lr)
epochs = 1000
batch_size = 32
class_weights = torch.Tensor([2.4433, 6.319, 8.3295, 7.33, 0.371,0.49, 0.7541, 0.8983]).cuda()
criterion = nn.CrossEntropyLoss(weight=class_weights)

# Setup dataloaders
train_csv = "train_norm.csv"
val_csv = "val_norm.csv"
train_dataset = CSVDataset(train_csv)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = CSVDataset(val_csv)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [6]:
for i, epoch in enumerate(range(epochs)):
    print(f"EPOCH {i} / {epochs - 1}")
    
    # Set to train() mode and run one epoch
    mlp.train()
    t_loss = mlp.train_model(train_loader, optimizer, criterion)
    print("Training loss =", t_loss / len(train_loader))

    # Set to eval() mode and run one epoch
    with torch.no_grad():
      mlp.eval()
      v_loss = mlp.validate_model(val_loader, criterion)
    print("Validation loss =", v_loss / len(val_loader))
    

EPOCH 0 / 999
Training loss = 2.4338079950083857
Validation loss = 2.3543388066084487
EPOCH 1 / 999
Training loss = 2.4084716102351313
Validation loss = 2.336502412091131
EPOCH 2 / 999
Training loss = 2.3897207832854725
Validation loss = 2.320874281551527
EPOCH 3 / 999
Training loss = 2.3815991969212242
Validation loss = 2.314799453901208
EPOCH 4 / 999
Training loss = 2.377018585153248
Validation loss = 2.3138343147609546
EPOCH 5 / 999
Training loss = 2.3719577944797017
Validation loss = 2.3140679546024487
EPOCH 6 / 999
Training loss = 2.3838942569235098
Validation loss = 2.314758026081583
EPOCH 7 / 999
Training loss = 2.377754322860552
Validation loss = 2.31509848781254
EPOCH 8 / 999
Training loss = 2.3755085001821103
Validation loss = 2.315525277801182
EPOCH 9 / 999
Training loss = 2.3684334210727527
Validation loss = 2.316953876744146
EPOCH 10 / 999
Training loss = 2.369757153417753
Validation loss = 2.3155417494151904
EPOCH 11 / 999
Training loss = 2.371061510365942
Validation loss

KeyboardInterrupt: ignored

In [ ]:
# Optional save notebook to 